In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/pytorch_colab/rolling_and_plot.py .

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor, Lambda
from dataclasses import dataclass

In [ ]:
import numpy as np
import pandas as pd
from rolling_and_plot import *

# %reload_ext autoreload
# %autoreload 2R

In [ ]:
# Get cpu or gpu device for training.
device = torch.device("cpu") #"mps" for M1 macs
print(f"Using {device} device")

# TOC

* [Data Loading](#dload)

* [Models](#model)

## <a id="g">G class</a>

In [ ]:
@dataclass
class G:
#     split_time = None #for now, might be useless
    # sequence_length = 5
    num_features = 4 # delta_t, current, voltage, and soc at t-1
    window_size = 2048
    batch_size = 8
    epochs = 15

# Data Loader <a id="dload"></a>

In [ ]:
# from google.colab import files
file = pd.read_csv("/content/")

In [ ]:
data_plot(data = [file],
          title="OCV v SOC",
          x = ["test time (sec)"],
          y = ["soc"],
          markers = "lines",
          color = "darkorchid"
         )

In [ ]:
train = file[["delta t","current","voltage","soc" ]].loc[:107898]

test = file[["delta t","current","voltage","soc" ]].loc[107899:]

#try without delta t

In [ ]:
# np.array_split(train,len(train)//6) not applicable here

In [ ]:
train = rolling(train, G.window_size)
test = rolling(test, G.window_size)

In [ ]:
class BatterySet(Dataset):
    def __init__(self, dataframe):
            
        x, y = dataframe
    
        self.logits = torch.from_numpy(x).to(device)
        self.labels = torch.from_numpy(y).to(device)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (self.logits[idx], self.labels[idx])

In [ ]:
train_dataloader = BatterySet(train)
test_dataloader = BatterySet(test)

In [ ]:
train_dataloader = DataLoader(train_dataloader, batch_size=G.batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataloader, batch_size=G.batch_size, shuffle=True)

In [ ]:
for X,y in train_dataloader.dataset:
    print(f"Shape of X [window, features]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
for batch, (x,y) in enumerate(test_dataloader.dataset):
    print(batch,x,y)
    break

# Creating Models <a id="model"></a>

Go to [G class](#g)

In [ ]:
# # Get cpu or gpu device for training.
# device = torch.device("mps")
# print(f"Using {device} device")

# Define model
class LSTMNetwork(nn.Module):
    
    def __init__(self):
        super(LSTMNetwork, self).__init__()

        self.batch_norm1 = nn.BatchNorm1d(G.batch_size, momentum = 0.2)
        self.batch_norm2 = nn.BatchNorm1d(1, momentum = 0.15)
        self.lstm = nn.LSTM(G.num_features, G.batch_size, 2, batch_first = True, dropout = 0.1)
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.uniform_(param, a=0.001, b=0.01)
            elif 'weight_ih' in name:
                nn.init.kaiming_normal_(param, nonlinearity = "relu")
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)

        # self.linear_relu_stack = nn.Sequential(
        #     nn.Linear(G.batch_size, G.batch_size // 4),
        #     nn.ReLU(),
        #     nn.Linear(G.batch_size // 4, 1)
        #     )
        # for name, param in self.linear_relu_stack.named_parameters():
        #     if 'bias' in name:
        #         nn.init.uniform_(param, a=0.0001, b=0.999)
        #     elif 'weight' in name:
        #         nn.init.kaiming_normal_(param, nonlinearity = "relu")

        self.linear = nn.Linear(G.batch_size,1)
        for name, param in self.linear.named_parameters():
            if 'bias' in name:
                nn.init.uniform_(param, a=0.0001, b=0.001)
            elif 'weight' in name:
                nn.init.kaiming_normal_(param, nonlinearity = "relu")

#     def l2_normalize(self, x, dim = 1):
#         "apparently weight decay in the optimize functions does l2 regularization"
#         return nn.functional.normalize(x, p = 2.0 , dim = dim)
    
    def forward(self, x):
#         x = self.flatten(x)
        x_out, (h_n, c_n)  = self.lstm(x)
        for i in [x_out, h_n, c_n]:
            i = self.batch_norm1(i)

        # logits = self.batch_norm2(self.linear_relu_stack(x_out))
        out = self.linear(h_n[-1])

        return out

model = LSTMNetwork().to(device)
print(model)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (x,y) in enumerate(dataloader.dataset):
        #forward
        predict = model(x)
        loss = loss_fn(predict.reshape(y.shape), y)
        
        #backward
        optimizer.zero_grad() #resets the gradient graph, a pytorch shortcoming that is required
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        if batch % 2000 == 0:
            loss, current = loss.item(), batch
            print(f"loss: {loss:>7f}  [{current:5d}/{size:5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = size // G.batch_size
    test_loss, perc_error = 0,0
    counter = 0
    with torch.no_grad(): #doesnt update parameters (we are testing not training)
        for x,y in dataloader.dataset:
            predict = model(x)
            test_loss += loss_fn(predict, y).item()
            perc_error += torch.mean(torch.abs(predict - y) / y * 100, 0).mean(0)
            #### will have to change the way correct is computed!!
           
            counter += 1
            if counter % 2000 == 0:
                print(f"{counter} / {size} tested")
            
            
    test_loss /= num_batches
    perc_error /= size
    print(f"Test Error: \nAverage Accuracy: {100 - perc_error}%, Avg Loss: {test_loss:>8f}\n")
    return perc_error, test_loss

In [ ]:
loss_fn = nn.HuberLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = 0.35,
                             weight_decay=1e-6,)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", factor = 0.15, patience = 0, cooldown = 0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 2, gamma=0.4, verbose=False)

In [ ]:
for epoch in range(G.epochs):
    print(f"Epoch {epoch+1}\n----------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Completed")

In [ ]:
torch.save(model.state_dict(), "drive/MyDrive/pytorch_colab/model_state_dict.pth")
torch.save(optimizer.state_dict(), "drive/MyDrive/pytorch_colab/optimizer_state_dict.pth")

In [ ]:
with torch.no_grad():
    predict_1 = model(test_dataloader.dataset.logits[1500])
    print(predict_1, test_dataloader.dataset.labels[1500])